In [1]:
!pip install --upgrade langchain langgraph
!pip install --upgrade langchain-ollama

run "!curl https://ollama.ai/install.sh | sh" in the termina before going to next step

In [3]:
import subprocess
process = subprocess.Popen("ollama serve", shell=True)

2025/05/06 23:27:54 routes.go:1233: INFO server config env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_CONTEXT_LENGTH:4096 OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_KV_CACHE_TYPE: OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/root/.ollama/models OLLAMA_MULTIUSER_CACHE:false OLLAMA_NEW_ENGINE:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:0 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://* vscode-webview://* vscode-file://*] OLLAMA_SCHED_SPREAD:false ROCR_VISIBLE_DEVICES: http_proxy: https_proxy:

Run "!ollama pull <your model>" before going to next steps

I used phi-4 model for this notebook implementation

In [4]:
!ollama list

[GIN] 2025/05/06 - 23:28:09 | 200 |       76.36µs |       127.0.0.1 | HEAD     "/"
[GIN] 2025/05/06 - 23:28:09 | 200 |     674.785µs |       127.0.0.1 | GET      "/api/tags"
NAME           ID              SIZE      MODIFIED       
phi4:latest    ac896e5b8b34    9.1 GB    32 minutes ago    


In [7]:
from langchain import hub

select_prompt = hub.pull("hwchase17/self-discovery-select")
print("Self-Discovery Select Prompt:")
select_prompt.pretty_print()
print("Self-Discovery Select Response:")
adapt_prompt = hub.pull("hwchase17/self-discovery-adapt")
adapt_prompt.pretty_print()
structured_prompt = hub.pull("hwchase17/self-discovery-structure")
print("Self-Discovery Structured Prompt:")
structured_prompt.pretty_print()
reasoning_prompt = hub.pull("hwchase17/self-discovery-reasoning")
print("Self-Discovery Structured Response:")
reasoning_prompt.pretty_print()

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Self-Discovery Select Prompt:
Select several reasoning modules that are crucial to utilize in order to solve the given task:

All reasoning module descriptions:
{reasoning_modules}

Task: {task_description}

Select several modules are crucial for solving the task above:

Self-Discovery Select Response:
Rephrase and specify each reasoning module so that it better helps solving the task:

SELECTED module descriptions:
{selected_modules}

Task: {task_description}

Adapt each reasoning module description to better solve the task:



/usr/local/lib/python3.11/dist-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Self-Discovery Structured Prompt:
Operationalize the reasoning modules into a step-by-step reasoning plan in JSON format:

Here's an example:

Example task:

If you follow these instructions, do you return to the starting point? Always face forward. Take 1 step backward. Take 9 steps left. Take 2 steps backward. Take 6 steps forward. Take 4 steps forward. Take 4 steps backward. Take 3 steps right.

Example reasoning structure:

{
    "Position after instruction 1":
    "Position after instruction 2":
    "Position after instruction n":
    "Is final position the same as starting position":
}

Adapted module description:
{adapted_modules}

Task: {task_description}

Implement a reasoning structure for solvers to follow step-by-step and arrive at correct answer.

Note: do NOT actually arrive at a conclusion in this pass. Your job is to generate a PLAN so that in the future you can fill it out and arrive at the correct conclusion for tasks like this


/usr/local/lib/python3.11/dist-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Self-Discovery Structured Response:
Follow the step-by-step reasoning plan in JSON to correctly solve the task. Fill in the values following the keys by reasoning specifically about the task given. Do not simply rephrase the keys.
    
Reasoning Structure:
{reasoning_structure}

Task: {task_description}


In [10]:
from typing import Optional
from typing_extensions import TypedDict

from langchain_core.output_parsers import StrOutputParser
#from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama

from langgraph.graph import END, START, StateGraph


class SelfDiscoverState(TypedDict):
    reasoning_modules: str
    task_description: str
    selected_modules: Optional[str]
    adapted_modules: Optional[str]
    reasoning_structure: Optional[str]
    answer: Optional[str]


model = ChatOllama(model="phi4")


def select(inputs):
    select_chain = select_prompt | model | StrOutputParser()
    return {"selected_modules": select_chain.invoke(inputs)}


def adapt(inputs):
    adapt_chain = adapt_prompt | model | StrOutputParser()
    return {"adapted_modules": adapt_chain.invoke(inputs)}


def structure(inputs):
    structure_chain = structured_prompt | model | StrOutputParser()
    return {"reasoning_structure": structure_chain.invoke(inputs)}


def reason(inputs):
    reasoning_chain = reasoning_prompt | model | StrOutputParser()
    return {"answer": reasoning_chain.invoke(inputs)}


graph = StateGraph(SelfDiscoverState)
graph.add_node(select)
graph.add_node(adapt)
graph.add_node(structure)
graph.add_node(reason)
graph.add_edge(START, "select")
graph.add_edge("select", "adapt")
graph.add_edge("adapt", "structure")
graph.add_edge("structure", "reason")
graph.add_edge("reason", END)
app = graph.compile()

In [11]:
reasoning_modules = [
    "1. How could I devise an experiment to help solve that problem?",
    "2. Make a list of ideas for solving this problem, and apply them one by one to the problem to see if any progress can be made.",
    # "3. How could I measure progress on this problem?",
    "4. How can I simplify the problem so that it is easier to solve?",
    "5. What are the key assumptions underlying this problem?",
    "6. What are the potential risks and drawbacks of each solution?",
    "7. What are the alternative perspectives or viewpoints on this problem?",
    "8. What are the long-term implications of this problem and its solutions?",
    "9. How can I break down this problem into smaller, more manageable parts?",
    "10. Critical Thinking: This style involves analyzing the problem from different perspectives, questioning assumptions, and evaluating the evidence or information available. It focuses on logical reasoning, evidence-based decision-making, and identifying potential biases or flaws in thinking.",
    "11. Try creative thinking, generate innovative and out-of-the-box ideas to solve the problem. Explore unconventional solutions, thinking beyond traditional boundaries, and encouraging imagination and originality.",
    # "12. Seek input and collaboration from others to solve the problem. Emphasize teamwork, open communication, and leveraging the diverse perspectives and expertise of a group to come up with effective solutions.",
    "13. Use systems thinking: Consider the problem as part of a larger system and understanding the interconnectedness of various elements. Focuses on identifying the underlying causes, feedback loops, and interdependencies that influence the problem, and developing holistic solutions that address the system as a whole.",
    "14. Use Risk Analysis: Evaluate potential risks, uncertainties, and tradeoffs associated with different solutions or approaches to a problem. Emphasize assessing the potential consequences and likelihood of success or failure, and making informed decisions based on a balanced analysis of risks and benefits.",
    # "15. Use Reflective Thinking: Step back from the problem, take the time for introspection and self-reflection. Examine personal biases, assumptions, and mental models that may influence problem-solving, and being open to learning from past experiences to improve future approaches.",
    "16. What is the core issue or problem that needs to be addressed?",
    "17. What are the underlying causes or factors contributing to the problem?",
    "18. Are there any potential solutions or strategies that have been tried before? If yes, what were the outcomes and lessons learned?",
    "19. What are the potential obstacles or challenges that might arise in solving this problem?",
    "20. Are there any relevant data or information that can provide insights into the problem? If yes, what data sources are available, and how can they be analyzed?",
    "21. Are there any stakeholders or individuals who are directly affected by the problem? What are their perspectives and needs?",
    "22. What resources (financial, human, technological, etc.) are needed to tackle the problem effectively?",
    "23. How can progress or success in solving the problem be measured or evaluated?",
    "24. What indicators or metrics can be used?",
    "25. Is the problem a technical or practical one that requires a specific expertise or skill set? Or is it more of a conceptual or theoretical problem?",
    "26. Does the problem involve a physical constraint, such as limited resources, infrastructure, or space?",
    "27. Is the problem related to human behavior, such as a social, cultural, or psychological issue?",
    "28. Does the problem involve decision-making or planning, where choices need to be made under uncertainty or with competing objectives?",
    "29. Is the problem an analytical one that requires data analysis, modeling, or optimization techniques?",
    "30. Is the problem a design challenge that requires creative solutions and innovation?",
    "31. Does the problem require addressing systemic or structural issues rather than just individual instances?",
    "32. Is the problem time-sensitive or urgent, requiring immediate attention and action?",
    "33. What kinds of solution typically are produced for this kind of problem specification?",
    "34. Given the problem specification and the current best solution, have a guess about other possible solutions."
    "35. Let’s imagine the current best solution is totally wrong, what other ways are there to think about the problem specification?"
    "36. What is the best way to modify this current best solution, given what you know about these kinds of problem specification?"
    "37. Ignoring the current best solution, create an entirely new solution to the problem."
    # "38. Let’s think step by step."
    "39. Let’s make a step by step plan and implement it with good notation and explanation.",
]


task_example = "Lisa has 10 apples. She gives 3 apples to her friend and then buys 5 more apples from the store. How many apples does Lisa have now?"

task_example = """This SVG path element <path d="M 55.57,80.69 L 57.38,65.80 M 57.38,65.80 L 48.90,57.46 M 48.90,57.46 L
45.58,47.78 M 45.58,47.78 L 53.25,36.07 L 66.29,48.90 L 78.69,61.09 L 55.57,80.69"/> draws a:
(A) circle (B) heptagon (C) hexagon (D) kite (E) line (F) octagon (G) pentagon(H) rectangle (I) sector (J) triangle"""

reasoning_modules_str = "\n".join(reasoning_modules)

for s in app.stream(
    {"task_description": task_example, "reasoning_modules": reasoning_modules_str}
):
    print(s)

time=2025-05-06T23:35:06.489Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32
time=2025-05-06T23:35:06.777Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32
time=2025-05-06T23:35:06.791Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32
time=2025-05-06T23:35:06.792Z level=WARN source=ggml.go:152 msg="key not found" key=phi3.vision.block_count default=0
time=2025-05-06T23:35:06.793Z level=WARN source=ggml.go:152 msg="key not found" key=phi3.attention.key_length default=128
time=2025-05-06T23:35:06.793Z level=WARN source=ggml.go:152 msg="key not found" key=phi3.attention.value_length default=128
time=2025-05-06T23:35:06.793Z level=INFO source=sched.go:754 msg="new model will fit in available VRAM in single GPU, loading" model=/root/.ollama/models/blobs/sha256-fd7b6731c33c57f61767612f56517460ec2d1e2e5a3f0163e0eb3d8d8cb5df20 gpu=GPU-ffe57e60-6d09-9154-8a7e-65a7cfcd290f parallel=2 available

[GIN] 2025/05/06 - 23:35:33 | 200 | 27.511502941s |       127.0.0.1 | POST     "/api/chat"
{'select': {'selected_modules': 'To solve the task of identifying what shape the given SVG path element draws, we need to focus on reasoning modules that help analyze and interpret geometric data effectively. Here are the selected reasoning modules:\n\n1. **9: How can I break down this problem into smaller, more manageable parts?**\n   - This module helps us decompose the complex path into its constituent line segments. By examining each segment individually (e.g., `M 55.57,80.69 L 57.38,65.80`), we can better understand how they connect to form a shape.\n\n2. **16: What is the core issue or problem that needs to be addressed?**\n   - The core issue here is determining what geometric shape is formed by the path commands given in the SVG element.\n\n3. **29: Is the problem an analytical one that requires data analysis, modeling, or optimization techniques?**\n   - This module is relevant because w

time=2025-05-06T23:35:34.001Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32


[GIN] 2025/05/06 - 23:36:02 | 200 | 28.078107124s |       127.0.0.1 | POST     "/api/chat"
{'adapt': {'adapted_modules': 'To effectively determine the shape that an SVG path element draws, we can refine our reasoning modules as follows:\n\n1. **Decomposition of Path Elements**\n   - This refined module focuses on breaking down the complex SVG path into its basic components: line segments defined by commands like `M` (move to) and `L` (line to). By analyzing each command individually (e.g., `M 55.57,80.69 L 57.38,65.80`), we can map out how these segments connect at specific coordinates, which helps us visualize the overall structure of the shape.\n\n2. **Identification of Core Geometric Problem**\n   - The core problem here is identifying the geometric figure formed by the sequence of path commands in the SVG element. This involves recognizing patterns and relationships between the points specified by the path data to determine what shape they collectively represent.\n\n3. **Analytical

time=2025-05-06T23:36:02.084Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32


[GIN] 2025/05/06 - 23:36:48 | 200 | 46.362963986s |       127.0.0.1 | POST     "/api/chat"
{'structure': {'reasoning_structure': 'To create a structured reasoning plan for determining the shape drawn by an SVG path element, we will outline a step-by-step approach using JSON format. This plan will guide solvers through analyzing the SVG path data to identify the geometric figure.\n\n```json\n{\n    "Step 1: Decomposition of Path Elements": {\n        "description": "Break down the SVG path into its basic components.",\n        "actions": [\n            "Identify each command (e.g., M, L) and extract coordinates.",\n            "List all distinct line segments defined by these commands."\n        ],\n        "output": "A list of line segments with their start and end coordinates."\n    },\n    \n    "Step 2: Identification of Core Geometric Problem": {\n        "description": "Determine the geometric figure formed by the path elements.",\n        "actions": [\n            "Identify patte

time=2025-05-06T23:36:48.450Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32


[GIN] 2025/05/06 - 23:38:01 | 200 |         1m12s |       127.0.0.1 | POST     "/api/chat"
{'reason': {'answer': 'To determine the shape drawn by the SVG path element, we will follow the structured reasoning plan outlined in JSON format.\n\n```json\n{\n    "Step 1: Decomposition of Path Elements": {\n        "description": "Break down the SVG path into its basic components.",\n        "actions": [\n            "Identify each command (e.g., M, L) and extract coordinates.",\n            "List all distinct line segments defined by these commands."\n        ],\n        "output": "A list of line segments with their start and end coordinates: \\n1. M 55.57,80.69 to L 57.38,65.80\\n2. M 57.38,65.80 to L 48.90,57.46\\n3. M 48.90,57.46 to L 45.58,47.78\\n4. M 45.58,47.78 to L 53.25,36.07\\n5. M 53.25,36.07 to L 66.29,48.90\\n6. M 66.29,48.90 to L 78.69,61.09\\n7. M 78.69,61.09 to L 55.57,80.69"\n        }\n    },\n    \n    "Step 2: Identification of Core Geometric Problem": {\n        "descrip

In [12]:
task_example = "Lisa has 10 apples. She gives 3 apples to her friend and then buys 5 more apples from the store. How many apples does Lisa have now?"

for s in app.stream(
    {"task_description": task_example, "reasoning_modules": reasoning_modules_str}
):
    print(s)

time=2025-05-06T23:39:24.255Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32


[GIN] 2025/05/06 - 23:39:55 | 200 | 31.237188109s |       127.0.0.1 | POST     "/api/chat"
{'select': {'selected_modules': "To solve the given task about Lisa's apples, we can identify a few reasoning modules that will help us systematically approach and solve this problem:\n\n1. **How can I simplify the problem so that it is easier to solve?**  \n   - This module helps in breaking down the problem into simpler arithmetic operations: subtracting the 3 apples given away and adding the 5 apples bought.\n\n2. **What is the core issue or problem that needs to be addressed?**  \n   - The core issue here involves understanding the sequence of transactions (giving away and buying) and calculating the final number of apples Lisa has.\n\n3. **How can I break down this problem into smaller, more manageable parts?**  \n   - This module allows us to separately handle each transaction: first subtracting 3 from 10, then adding 5 to the result.\n\n4. **Are there any potential obstacles or challenges 

time=2025-05-06T23:39:55.497Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32


[GIN] 2025/05/06 - 23:40:31 | 200 | 36.149314861s |       127.0.0.1 | POST     "/api/chat"
{'adapt': {'adapted_modules': "To systematically approach and solve the problem of calculating how many apples Lisa has after giving some away and buying more, we can employ specific reasoning modules that guide us through each step:\n\n1. **How do I simplify the arithmetic operations involved in this scenario?**  \n   - This module focuses on reducing complexity by converting the sequence of actions into basic mathematical steps: first subtracting 3 apples (given away) from the initial count and then adding 5 apples (bought).\n\n2. **What is the primary objective or problem to be solved here?**  \n   - The main goal is to determine Lisa's final apple count by accurately considering both the subtraction of apples she gives away and the addition of those she buys.\n\n3. **How can I decompose this scenario into discrete, manageable steps?**  \n   - This module encourages breaking down the problem i

time=2025-05-06T23:40:31.650Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32


[GIN] 2025/05/06 - 23:41:19 | 200 | 48.146433467s |       127.0.0.1 | POST     "/api/chat"
{'structure': {'reasoning_structure': 'To operationalize the reasoning process into a structured plan, we\'ll create a JSON format that breaks down the task of determining how many apples Lisa has after her transactions. This structure will guide solvers through each step systematically.\n\n```json\n{\n    "Initial Setup": {\n        "Description": "Start by identifying Lisa\'s initial number of apples.",\n        "Action": "Note the initial count of apples.",\n        "Current Position": "Lisa starts with 10 apples."\n    },\n    "Step 1: Subtract Apples Given Away",\n    {\n        "Description": "Subtract the number of apples Lisa gives to her friend.",\n        "Action": "Calculate the remaining apples after giving away 3 apples.",\n        "Calculation": "Initial apples - Apples given away",\n        "Current Position": "Lisa has X apples after giving away."\n    },\n    "Step 2: Add Apples 

time=2025-05-06T23:41:19.810Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32


{'reason': {'answer': 'To solve this task using a structured reasoning plan, we will follow each step as outlined in the JSON format:\n\n```json\n{\n    "Initial Setup": {\n        "Description": "Start by identifying Lisa\'s initial number of apples.",\n        "Action": "Note the initial count of apples.",\n        "Current Position": "Lisa starts with 10 apples."\n    },\n    "Step 1: Subtract Apples Given Away": {\n        "Description": "Subtract the number of apples Lisa gives to her friend.",\n        "Action": "Calculate the remaining apples after giving away 3 apples.",\n        "Calculation": "Initial apples (10) - Apples given away (3)",\n        "Current Position": "Lisa has 7 apples after giving away."\n    },\n    "Step 2: Add Apples Bought": {\n        "Description": "Add the number of apples Lisa buys from the store.",\n        "Action": "Calculate the total apples after buying 5 more.",\n        "Calculation": "Remaining apples (7) + Apples bought (5)",\n        "Curre

In [13]:
task_example = "we have 5 crows on a branch. 3 of them flew away, 2 came back and 3 new crows joined. How many crows are on a branch?"

for s in app.stream(
    {"task_description": task_example, "reasoning_modules": reasoning_modules_str}
):
    print(s)

time=2025-05-06T23:50:59.973Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32
time=2025-05-06T23:51:00.232Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32
time=2025-05-06T23:51:00.245Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32
time=2025-05-06T23:51:00.246Z level=WARN source=ggml.go:152 msg="key not found" key=phi3.vision.block_count default=0
time=2025-05-06T23:51:00.246Z level=WARN source=ggml.go:152 msg="key not found" key=phi3.attention.key_length default=128
time=2025-05-06T23:51:00.246Z level=WARN source=ggml.go:152 msg="key not found" key=phi3.attention.value_length default=128
time=2025-05-06T23:51:00.246Z level=INFO source=sched.go:754 msg="new model will fit in available VRAM in single GPU, loading" model=/root/.ollama/models/blobs/sha256-fd7b6731c33c57f61767612f56517460ec2d1e2e5a3f0163e0eb3d8d8cb5df20 gpu=GPU-ffe57e60-6d09-9154-8a7e-65a7cfcd290f parallel=2 available

[GIN] 2025/05/06 - 23:51:31 | 200 |  31.67124938s |       127.0.0.1 | POST     "/api/chat"
{'select': {'selected_modules': "To solve the given task about calculating the number of crows on a branch after certain changes occur, we can utilize several reasoning modules to ensure clarity and correctness in our approach:\n\n1. **4: How can I simplify the problem so that it is easier to solve?**\n   - Simplifying involves breaking down the sequence of events into manageable steps: initially, there are 5 crows; then adjustments based on those that flew away, came back, or joined.\n\n2. **9: How can I break down this problem into smaller, more manageable parts?**\n   - Break it down step by step:\n     - Start with the initial count of crows.\n     - Subtract the number of crows that flew away.\n     - Add the number of crows that returned.\n     - Add the new crows that joined.\n\n3. **16: What is the core issue or problem that needs to be addressed?**\n   - Identify the main task: determini

time=2025-05-06T23:51:31.647Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32


[GIN] 2025/05/06 - 23:52:03 | 200 | 32.171140992s |       127.0.0.1 | POST     "/api/chat"
{'adapt': {'adapted_modules': "To effectively solve the problem of calculating the number of crows on a branch after specific changes, we can refine each reasoning module as follows:\n\n1. **Simplification: Breaking Down Complexity**\n   - Focus on decomposing the sequence of events into straightforward steps. Begin with an initial count of 5 crows and then apply sequential adjustments for those that flew away, returned, or joined.\n\n2. **Decomposition: Step-by-Step Approach**\n   - Divide the problem into clear steps:\n     - Start with the initial number of crows.\n     - Subtract the crows that flew away.\n     - Add back the crows that returned.\n     - Include any new crows that arrived.\n\n3. **Core Problem Identification: Defining Objectives**\n   - Pinpoint the primary goal: accurately calculate the final count of crows on the branch after all specified changes have been accounted for.\n

time=2025-05-06T23:52:03.819Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32


[GIN] 2025/05/06 - 23:53:03 | 200 | 59.690769122s |       127.0.0.1 | POST     "/api/chat"
{'structure': {'reasoning_structure': 'To implement a structured reasoning plan, we\'ll break down the task into clear steps using a JSON format. This will guide solvers through each stage of calculating the final number of crows on the branch:\n\n```json\n{\n    "Initial Setup": {\n        "description": "Start with an initial count of crows.",\n        "initial_count": 5,\n        "step_details": "Begin with 5 crows on the branch."\n    },\n    "Step-by-Step Adjustments": [\n        {\n            "step_number": 1,\n            "action": "Subtract",\n            "description": "Account for crows that flew away.",\n            "value_changed": -3,\n            "current_count_calculation": "5 - 3",\n            "resulting_count": 2\n        },\n        {\n            "step_number": 2,\n            "action": "Add",\n            "description": "Include the crows that returned.",\n            "value

time=2025-05-06T23:53:03.515Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32


[GIN] 2025/05/06 - 23:54:00 | 200 | 56.576538762s |       127.0.0.1 | POST     "/api/chat"
{'reason': {'answer': 'To solve this task using a structured reasoning plan in JSON format, let\'s break it down step-by-step:\n\n```json\n{\n    "Initial Setup": {\n        "description": "Start with an initial count of crows.",\n        "initial_count": 5,\n        "step_details": "Begin with 5 crows on the branch."\n    },\n    "Step-by-Step Adjustments": [\n        {\n            "step_number": 1,\n            "action": "Subtract",\n            "description": "Account for crows that flew away.",\n            "value_changed": -3,\n            "current_count_calculation": "5 - 3",\n            "resulting_count": 2\n        },\n        {\n            "step_number": 2,\n            "action": "Add",\n            "description": "Include the crows that returned.",\n            "value_changed": 2,\n            "current_count_calculation": "2 + 2",\n            "resulting_count": 4\n        },\n      

In [14]:
task_example = "Alice has four brothers and she also has a sister. How many sisters does Alice's brother have?"

for s in app.stream(
    {"task_description": task_example, "reasoning_modules": reasoning_modules_str}
):
    print(s)

time=2025-05-06T23:57:50.476Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32


[GIN] 2025/05/06 - 23:58:18 | 200 | 28.152591059s |       127.0.0.1 | POST     "/api/chat"
{'select': {'selected_modules': "To solve the task of determining how many sisters Alice's brother has, we can utilize the following reasoning modules:\n\n1. **What is the core issue or problem that needs to be addressed?**\n   - This module helps clarify the main question: How many sisters does each of Alice’s brothers have?\n\n2. **Is the problem related to human behavior, such as a social, cultural, or psychological issue?**\n   - This module confirms that the problem involves understanding family relationships, which is a straightforward logical deduction rather than requiring deep insight into human behavior.\n\n3. **What are the key assumptions underlying this problem?**\n   - This module helps identify any implicit assumptions, such as each brother having the same siblings as Alice.\n\n4. **Critical Thinking: Analyze the problem from different perspectives, question assumptions, and evalua

time=2025-05-06T23:58:18.634Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32


[GIN] 2025/05/06 - 23:58:45 | 200 | 27.034172697s |       127.0.0.1 | POST     "/api/chat"
{'adapt': {'adapted_modules': "To accurately determine how many sisters each of Alice's brothers has, we can refine our approach with specific reasoning modules:\n\n1. **Clarify the Specific Inquiry:**\n   - This module focuses on pinpointing the precise question: How many sisters does each of Alice’s brothers have? It ensures we are addressing the correct aspect of the family structure.\n\n2. **Identify Logical Relationships:**\n   - Here, we establish that the problem involves understanding familial connections rather than complex human behaviors. The task is to logically deduce sibling relationships based on shared parentage.\n\n3. **Assess Underlying Assumptions:**\n   - This module helps us recognize implicit assumptions, such as all siblings having the same parents and thus sharing the same set of brothers and sisters.\n\n4. **Apply Logical Deduction:**\n   - This involves analyzing the inf

time=2025-05-06T23:58:45.667Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32


[GIN] 2025/05/06 - 23:59:21 | 200 | 36.206233662s |       127.0.0.1 | POST     "/api/chat"
{'structure': {'reasoning_structure': 'Here\'s a step-by-step reasoning plan structured in JSON format designed for solvers to follow when determining how many sisters each of Alice\'s brothers has:\n\n```json\n{\n    "Step 1: Clarify the Specific Inquiry": {\n        "description": "Identify the precise question to be answered.",\n        "details": [\n            "Determine what needs to be found: The number of sisters that each of Alice’s brothers has."\n        ]\n    },\n    "Step 2: Identify Logical Relationships": {\n        "description": "Establish the logical framework concerning family relationships.",\n        "details": [\n            "Recognize that the task involves analyzing familial connections based on shared parentage."\n        ]\n    },\n    "Step 3: Assess Underlying Assumptions": {\n        "description": "Identify any implicit assumptions within the scenario.",\n        "d

time=2025-05-06T23:59:21.879Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32


[GIN] 2025/05/06 - 23:59:59 | 200 | 37.991971128s |       127.0.0.1 | POST     "/api/chat"
{'reason': {'answer': '```json\n{\n    "Step 1: Clarify the Specific Inquiry": {\n        "description": "Identify the precise question to be answered.",\n        "details": [\n            "Determine what needs to be found: The number of sisters that each of Alice’s brothers has."\n        ]\n    },\n    "Step 2: Identify Logical Relationships": {\n        "description": "Establish the logical framework concerning family relationships.",\n        "details": [\n            "Recognize that the task involves analyzing familial connections based on shared parentage. Alice, her four brothers, and her sister are siblings because they have the same parents."\n        ]\n    },\n    "Step 3: Assess Underlying Assumptions": {\n        "description": "Identify any implicit assumptions within the scenario.",\n        "details": [\n            "Assume all siblings share the same parents, thereby having the s

In [15]:
task_example =  "Alice has 4 sisters and she also has 1 brother. How many sisters does Alice's brother have? Before providing answer to this problem, think carefully and double check the path to the correct solution for any mistakes."
for s in app.stream(
    {"task_description": task_example, "reasoning_modules": reasoning_modules_str}
):
    print(s)

time=2025-05-07T00:03:43.778Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32


[GIN] 2025/05/07 - 00:04:13 | 200 | 30.217369864s |       127.0.0.1 | POST     "/api/chat"
{'select': {'selected_modules': "To solve the given task about how many sisters Alice's brother has, we need to focus on understanding relationships and ensuring clarity in logical reasoning. Here are the key reasoning modules that would be useful:\n\n1. **What is the core issue or problem that needs to be addressed? (16)**\n   - Understanding the core question: We need to determine how many sisters Alice’s brother has.\n\n2. **How can I break down this problem into smaller, more manageable parts? (9)**\n   - Breaking down the relationships: Identify Alice's siblings and consider her brother’s perspective in terms of family structure.\n\n3. **What are the key assumptions underlying this problem? (5)**\n   - Clarifying assumptions: We assume that all of Alice’s sisters are also her brother’s sisters, as they share the same parents.\n\n4. **Critical Thinking: Analyzing the problem from different pe

time=2025-05-07T00:04:13.998Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32


[GIN] 2025/05/07 - 00:04:39 | 200 | 25.226669704s |       127.0.0.1 | POST     "/api/chat"
{'adapt': {'adapted_modules': "To effectively address the task of determining how many sisters Alice's brother has, we need to focus on understanding family relationships and employing logical reasoning. Here are refined descriptions of the key reasoning modules:\n\n1. **Identify the Core Question (16)**\n   - Focus on the central inquiry: Determine the number of sisters that Alice’s brother possesses.\n\n2. **Decompose the Problem into Manageable Parts (9)**\n   - Break down family relationships: Identify all siblings of Alice and consider their relationship to her brother, focusing specifically on those who are his sisters.\n\n3. **Clarify Underlying Assumptions (5)**\n   - Establish assumptions: Assume that any sister of Alice is also a sister to her brother, given they share the same parents.\n\n4. **Apply Critical Thinking (10)**\n   - Analyze perspectives and verify logic: Ensure there are 

time=2025-05-07T00:04:39.226Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32


[GIN] 2025/05/07 - 00:05:17 | 200 | 38.198811297s |       127.0.0.1 | POST     "/api/chat"
{'structure': {'reasoning_structure': 'To address the task of determining how many sisters Alice\'s brother has, we will develop a structured reasoning plan using JSON format. This plan will guide solvers through each logical step needed to reach a conclusion.\n\n```json\n{\n    "Identify the Core Question": "Determine the number of sisters that Alice’s brother possesses.",\n    \n    "Decompose the Problem into Manageable Parts": [\n        {\n            "Step 1": "Identify all siblings of Alice. From the problem, we know Alice has 4 sisters and 1 brother."\n        },\n        {\n            "Step 2": "Focus on identifying who among these siblings are sisters to her brother."\n        }\n    ],\n    \n    "Clarify Underlying Assumptions": [\n        {\n            "Assumption 1": "All of Alice\'s sisters are also her brother’s sisters, as they share the same parents."\n        },\n        {\n 

time=2025-05-07T00:05:17.429Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32


{'reason': {'answer': 'To solve the task of determining how many sisters Alice\'s brother has, let\'s follow a step-by-step reasoning plan structured in JSON format:\n\n```json\n{\n    "Identify the Core Question": "Determine the number of sisters that Alice’s brother possesses.",\n    \n    "Decompose the Problem into Manageable Parts": [\n        {\n            "Step 1": "Identify all siblings of Alice. From the problem, we know Alice has 4 sisters and 1 brother."\n        },\n        {\n            "Step 2": "Focus on identifying who among these siblings are sisters to her brother."\n        }\n    ],\n    \n    "Clarify Underlying Assumptions": [\n        {\n            "Assumption 1": "All of Alice\'s sisters are also her brother’s sisters, as they share the same parents."\n        },\n        {\n            "Assumption 2": "The relationship structure implies that siblings within a nuclear family share parents equally unless otherwise specified (e.g., half-siblings)."\n        }\n